In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

data1 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/Human_ifgn.h5ad') # Please modify this to your address.
data1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AnnData object with n_obs × n_vars = 114175 × 40
    obs: 'sample', 'cluster_o', 'cluster_s'

In [2]:
data1.obs['cluster_s']

0         Neutrophil
1         Neutrophil
2              CD4 T
3              CD4 T
4         Neutrophil
             ...    
114170         CD4 T
114171    Neutrophil
114172    Neutrophil
114173    Neutrophil
114174    Neutrophil
Name: cluster_s, Length: 114175, dtype: category
Categories (7, object): ['B', 'Basophil', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil']

In [3]:
data1.var_names

Index(['index', 'CD235a', 'CD45', 'STAT3', 'CD61', 'CD7', 'CD33', 'CD11c',
       'MAPKAPK2', 'CREB', 'PLCg2', 'STAT1', 'CD123', 'STAT5', 'CD14',
       'Erk1.2', 'CD11b', 'p38', 'CD8', 'CD4', 'CD3', 'CD66', 'CD16',
       'Zap70.Syk', 'TBK1', 'CD1c', 'IkBa', 'BDCA3', 'STAT6', 'CD45RA',
       'Fox.P3', 'CD161', 'Ki67', 'STAT4', 'CCR7', 'X4EBP1', 'CD19', 'IgM',
       'CD56', 'HLA.DR'],
      dtype='object')

In [ ]:
adata1 = data1.copy()
print(adata1)


AnnData object with n_obs × n_vars = 114175 × 40
    obs: 'sample', 'cluster_o', 'cluster_s'


In [5]:
data2 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/Human_with_h1n1.h5ad') # Please modify this to your address.
data2

AnnData object with n_obs × n_vars = 102147 × 41
    obs: 'sample', 'cluster_o', 'cluster_s'

In [ ]:
adata2 = data2.copy()
print(adata2)

AnnData object with n_obs × n_vars = 102147 × 41
    obs: 'sample', 'cluster_o', 'cluster_s'


In [7]:
var_names_union = adata1.var_names.intersection(adata2.var_names)
print(var_names_union)
print(var_names_union.shape)

Index(['CD235a', 'CD45', 'STAT3', 'CD61', 'CD7', 'CD33', 'CD11c', 'MAPKAPK2',
       'CREB', 'PLCg2', 'STAT1', 'CD123', 'STAT5', 'CD14', 'Erk1.2', 'CD11b',
       'p38', 'CD8', 'CD4', 'CD3', 'CD66', 'CD16', 'Zap70.Syk', 'TBK1', 'CD1c',
       'IkBa', 'BDCA3', 'STAT6', 'CD45RA', 'Fox.P3', 'CD161', 'Ki67', 'STAT4',
       'CCR7', 'X4EBP1', 'CD19', 'IgM', 'CD56', 'HLA.DR'],
      dtype='object')
(39,)


In [8]:
new_index = var_names_union.difference(['index'])
print(new_index)
print(new_index.shape)

Index(['BDCA3', 'CCR7', 'CD11b', 'CD11c', 'CD123', 'CD14', 'CD16', 'CD161',
       'CD19', 'CD1c', 'CD235a', 'CD3', 'CD33', 'CD4', 'CD45', 'CD45RA',
       'CD56', 'CD61', 'CD66', 'CD7', 'CD8', 'CREB', 'Erk1.2', 'Fox.P3',
       'HLA.DR', 'IgM', 'IkBa', 'Ki67', 'MAPKAPK2', 'PLCg2', 'STAT1', 'STAT3',
       'STAT4', 'STAT5', 'STAT6', 'TBK1', 'X4EBP1', 'Zap70.Syk', 'p38'],
      dtype='object')
(39,)


In [9]:
adata1 = adata1[:, new_index]
adata2 = adata2[:, new_index]

In [10]:
celltype = np.intersect1d(adata1.obs['cluster_s'].values, adata2.obs['cluster_s'].values)
celltype

array(['B', 'Basophil', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil'],
      dtype=object)

In [11]:
x1 = adata1.X
y1 = adata1.obs['cluster_s']

x, y = None, None
celltype_num = len(celltype)
cells_sub = []

for i in range(celltype_num):
    cells_sub.append(x1[np.array(y1 == celltype[i]), :])

for i in range(20000):

    # Create fractions for available celltypes
    fracs = np.random.rand(celltype_num)
    fracs_sum = np.sum(fracs)
    fracs = np.divide(fracs, fracs_sum)

    samp_fracs = np.multiply(fracs, 200)
    samp_fracs = np.array(list(map(round, samp_fracs)))

    artificial_samples = None

    for j in range(celltype_num):
        cells_fraction = np.random.randint(0, cells_sub[j].shape[0], samp_fracs[j])
        cells_choice = cells_sub[j][cells_fraction, :]
        if cells_choice.shape[0] > 0:
            artificial_samples = np.concatenate((artificial_samples, cells_choice)) if artificial_samples is not None else cells_choice

    cell = artificial_samples.mean(0).reshape(-1, 1)
    cell_y = (samp_fracs/samp_fracs.sum()).reshape(-1, 1)
    
    x = np.concatenate((x, cell), axis=1) if x is not None else cell
    y = np.concatenate((y, cell_y), axis=1) if y is not None else cell_y
    if i % 1000 == 0: print(i)

print(x.shape, y.shape)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
(39, 20000) (7, 20000)


In [12]:
df = pd.DataFrame(x.T, columns=adata1.var_names)
df

,BDCA3,CCR7,CD11b,CD11c,CD123,CD14,CD16,CD161,CD19,CD1c,...,PLCg2,STAT1,STAT3,STAT4,STAT5,STAT6,TBK1,X4EBP1,Zap70.Syk,p38
0,1.976014,3.496465,3.677431,1.388592,1.665742,1.402731,2.938402,2.972122,0.679581,0.436103,...,0.956347,1.733485,2.660052,1.175032,1.705171,0.861241,1.201784,1.175916,0.694351,1.379448
1,2.110822,3.714478,3.818212,1.494602,1.131280,1.975849,2.835386,3.117796,1.015196,0.523375,...,1.171375,1.829828,2.966349,1.267544,1.648616,0.871251,1.161758,1.473204,0.828849,1.608912
2,2.124038,3.529373,3.683571,1.393457,1.678946,1.568973,2.678866,2.977823,1.048754,0.546356,...,1.085354,1.593576,2.892730,1.208884,1.756825,0.849808,1.326560,1.436099,0.753226,1.538161
3,2.442647,3.775232,3.691114,1.580592,1.508232,2.108091,2.587967,3.186419,1.647210,0.561382,...,1.166249,1.839184,2.860152,1.332269,1.764966,0.981577,1.377034,1.787523,0.845017,1.706004
4,1.961539,3.584361,3.899525,1.512208,1.507542,1.715739,2.803661,2.902222,0.863746,0.462926,...,1.039172,1.903648,2.735616,1.153344,1.762233,0.843226,1.206046,1.355757,0.774231,1.489017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1.676778,3.435889,3.141622,1.035128,0.826035,1.442984,2.375117,2.833550,0.597111,0.360274,...,0.936291,1.755879,2.773922,0.962824,1.501840,0.718887,1.047703,1.037428,0.710815,1.464638
19996,2.068669,3.541468,3.787334,1.619656,1.437431,1.821006,2.971080,3.008850,0.697169,0.439110,...,1.082382,1.871114,2.772026,1.155868,1.778542,0.848101,1.228067,1.320950,0.707322,1.485199
19997,2.144031,3.652555,3.384645,1.407717,1.089986,1.909695,2.322278,3.143910,1.097282,0.533770,...,1.144219,1.757042,2.765118,1.208040,1.590247,0.867803,1.225379,1.522363,0.767910,1.593826
19998,2.265050,3.649319,4.353780,1.979120,1.525793,2.112119,3.463431,3.117624,0.966414,0.548497,...,1.118448,2.063899,2.804422,1.280584,1.808127,0.839633,1.230693,1.495308,0.822828,1.449587


In [13]:
adata_0 = sc.AnnData(df)
for i in range(len(celltype)):
    adata_0.obs[celltype[i]] = y[i].T
adata_0.obs['batch'] = np.array(['0'] * adata_0.shape[0])
adata_0.uns['cell_types'] = celltype

adata_0

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 20000 × 39
    obs: 'B', 'Basophil', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil', 'batch'
    uns: 'cell_types'

In [14]:
x2 = adata2.X
y2 = adata2.obs['cluster_s']

x, y = None, None
celltype_num = len(celltype)
cells_sub = []

for i in range(celltype_num):
    cells_sub.append(x2[np.array(y2 == celltype[i]), :])

for i in range(4000):

    # Create fractions for available celltypes
    fracs = np.random.rand(celltype_num)
    fracs_sum = np.sum(fracs)
    fracs = np.divide(fracs, fracs_sum)

    samp_fracs = np.multiply(fracs, 200)
    samp_fracs = np.array(list(map(round, samp_fracs)))

    artificial_samples = None

    for j in range(celltype_num):
        cells_fraction = np.random.randint(0, cells_sub[j].shape[0], samp_fracs[j])
        cells_choice = cells_sub[j][cells_fraction, :]
        if cells_choice.shape[0] > 0:
            artificial_samples = np.concatenate((artificial_samples, cells_choice)) if artificial_samples is not None else cells_choice

    cell = artificial_samples.mean(0).reshape(-1, 1)
    cell_y = (samp_fracs/samp_fracs.sum()).reshape(-1, 1)
    
    x = np.concatenate((x, cell), axis=1) if x is not None else cell
    y = np.concatenate((y, cell_y), axis=1) if y is not None else cell_y
    if i % 1000 == 0: print(i)

print(x.shape, y.shape)

0
1000
2000
3000
(39, 4000) (7, 4000)


In [15]:
df2 = pd.DataFrame(x.T, columns=adata2.var_names)
df2

,BDCA3,CCR7,CD11b,CD11c,CD123,CD14,CD16,CD161,CD19,CD1c,...,PLCg2,STAT1,STAT3,STAT4,STAT5,STAT6,TBK1,X4EBP1,Zap70.Syk,p38
0,1.708541,3.023689,3.757212,1.243466,1.388030,1.488598,2.161887,2.914919,1.035627,0.407431,...,0.670058,0.681480,2.477629,1.268666,2.280759,1.130799,0.861783,1.277210,0.985213,1.493435
1,1.973900,3.097238,3.846254,1.397160,1.208866,1.695858,2.102065,3.126477,1.154120,0.423496,...,0.739530,0.680511,2.503870,1.329679,2.297611,1.170244,0.942347,1.419595,0.949422,1.524426
2,1.666600,2.830353,4.149364,1.121007,1.099118,1.377756,2.774472,3.053655,0.666008,0.360853,...,0.738366,0.636548,2.510277,1.249870,2.256043,1.113876,0.886617,1.149373,0.989115,1.485256
3,1.811533,2.954212,4.211298,1.534706,1.335066,1.748261,2.456508,2.824361,1.250873,0.510169,...,0.745906,0.670760,2.576504,1.383581,2.300682,1.103317,0.924716,1.471465,1.075779,1.497436
4,1.804985,3.131691,3.132363,0.859571,1.642728,1.301053,1.722575,2.952547,1.308976,0.529593,...,0.748169,0.646319,2.436415,1.330408,2.358430,1.149453,0.917904,1.324982,0.969200,1.507378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1.833258,3.036320,3.265486,0.877139,1.194961,1.236280,2.186864,3.310397,1.465183,0.626424,...,0.706999,0.628833,2.422003,1.276904,2.226175,1.206793,0.993165,1.315920,0.904863,1.490681
3996,2.029217,3.163163,3.470486,1.348305,1.080277,1.657611,1.797921,3.248704,1.163944,0.406076,...,0.703650,0.603749,2.519263,1.313950,2.328766,1.201066,0.956053,1.361384,0.886426,1.513012
3997,1.874304,2.945449,4.038445,1.384973,1.115510,1.596603,2.443472,3.112700,1.091432,0.423576,...,0.664656,0.624553,2.479820,1.300794,2.241471,1.162072,0.929476,1.321198,0.938344,1.503252
3998,1.790067,3.024207,3.303670,0.853449,0.998086,1.255589,2.194682,3.268581,1.125296,0.538971,...,0.688898,0.616954,2.427068,1.273616,2.241062,1.152801,0.952469,1.249159,0.926452,1.500383


In [16]:
adata_1 = sc.AnnData(df2)
for i in range(len(celltype)):
    adata_1.obs[celltype[i]] = y[i].T
adata_1.obs['batch'] = np.array(['1'] * adata_1.shape[0])
adata_1.uns['cell_types'] = celltype

adata_1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 4000 × 39
    obs: 'B', 'Basophil', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil', 'batch'
    uns: 'cell_types'

In [17]:
adata_0.var_names_make_unique()
adata_1.var_names_make_unique()
adata = sc.concat((adata_0, adata_1))
adata.uns['cell_types'] = celltype
adata

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 24000 × 39
    obs: 'B', 'Basophil', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil', 'batch'
    uns: 'cell_types'

In [18]:
adata.write_h5ad('/public/home/syj/scpDeconv/dataset/mixup_data/Human_2_h1n1.h5ad') # Please modify this to your address.